In [70]:
from dagster import asset
from datetime import datetime

import plotly.express as px
import plotly.io as pio
import geopandas as gpd
import pandas as pd

import duckdb
import os

In [71]:
TAXI_ZONES_FILE_PATH = "data/raw/taxi_zones.csv"
TAXI_TRIPS_TEMPLATE_FILE_PATH = "data/raw/taxi_trips_{}.parquet"

TRIPS_BY_AIRPORT_FILE_PATH = "data/outputs/trips_by_airport.csv"
TRIPS_BY_WEEK_FILE_PATH = "data/outputs/trips_by_week.csv"
MANHATTAN_STATS_FILE_PATH = "data/staging/manhattan_stats.geojson"
MANHATTAN_MAP_FILE_PATH = "data/outputs/manhattan_map.png"

REQUEST_DESTINATION_TEMPLATE_FILE_PATH = "data/outputs/{}.png"

DATE_FORMAT = "%Y-%m-%d"

START_DATE = "2023-01-01"
END_DATE = "2023-04-01"


In [74]:
conn = duckdb.connect("data/staging/data.duckdb")
date_range = pd.date_range(start="03-01-2023", end='04-01-2023', freq='W-SUN')
result = pd.DataFrame()
for current_date in date_range:
    current_date_fmt = current_date.strftime(DATE_FORMAT)
    query = f"""
            SELECT 
                vendor_id, total_amount, trip_distance, passenger_count
            FROM trips
            WHERE date_trunc('week', pickup_datetime) = date_trunc('week', '{current_date_fmt}'::date)
        """
    data_for_week = conn.execute(query).fetch_df()
    data_for_week['period'] = current_date
    data_by_week = data_for_week.groupby('period').agg({
        "vendor_id": "count",
        "total_amount": "sum",
        "trip_distance": "sum",
        "passenger_count": "sum"
    }).rename({"vendor_id": "num_trips"})
    result = pd.concat([result, data_by_week])


In [75]:
result.tail(5)

,vendor_id,total_amount,trip_distance,passenger_count
period,,,,
2023-03-05,560895,15181854.49,2412606.42,739953.0
2023-03-12,774813,21192570.67,2739557.76,1019293.0
2023-03-19,787016,22096056.77,3079735.47,1044648.0
2023-03-26,734288,20675428.19,2935650.00,969859.0
